# 混合搜索
LangChain中的标准搜索通过向量相似性实现。然而，许多[向量存储](/docs/integrations/vectorstores/)实现（如Astra DB、ElasticSearch、Neo4J、AzureSearch、Qdrant等）还支持更高级的搜索方式，将向量相似性搜索与其他搜索技术（全文检索、BM25等）相结合。这种方式通常被称为“混合”搜索。
**第一步：确保您使用的向量存储支持混合搜索**
目前，LangChain 中尚未统一实现混合搜索的方法。每个向量存储库可能有自己的实现方式，通常通过 `similarity_search` 方法调用时传入的关键字参数来体现。
通过阅读文档或源代码，确定您所使用的向量存储是否支持混合搜索，如果支持，了解其使用方法。
**第二步：将该参数添加为链的可配置字段**
这将让你能够轻松调用链并在运行时配置任何相关标志。有关配置的更多信息，请参阅[此文档](/docs/how_to/configure)。
**步骤3：使用该可配置字段调用链**
现在，在运行时你可以使用可配置字段调用此链。
## 代码示例
让我们通过一个具体的代码示例来看看这是如何实现的。本例中，我们将使用 Astra DB 的 Cassandra/CQL 接口。
安装以下 Python 包：

In [ ]:
!pip install "cassio>=0.1.7"

获取[连接密钥](https://docs.datastax.com/en/astra/astra-db-vector/get-started/quickstart.html)。
初始化 cassio：

In [ ]:
import cassio

cassio.init(
    database_id="Your database ID",
    token="Your application token",
    keyspace="Your key space",
)

使用标准的[索引分析器](https://docs.datastax.com/en/astra/astra-db-vector/cql/use-analyzers-with-cql.html)创建Cassandra VectorStore。启用术语匹配需要索引分析器。

In [ ]:
from cassio.table.cql import STANDARD_ANALYZER
from langchain_community.vectorstores import Cassandra
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = Cassandra(
    embedding=embeddings,
    table_name="test_hybrid",
    body_index_options=[STANDARD_ANALYZER],
    session=None,
    keyspace=None,
)

vectorstore.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
    ]
)

如果我们执行标准的相似性搜索，会得到所有文档：

In [ ]:
vectorstore.as_retriever().invoke("What city did I visit last?")

[Document(page_content='In 2022, I visited New York'),
Document(page_content='In 2023, I visited Paris'),
Document(page_content='In 2021, I visited New Orleans')]

Astra DB 向量存储的 `body_search` 参数可用于对术语 `new` 进行搜索过滤。

In [ ]:
vectorstore.as_retriever(search_kwargs={"body_search": "new"}).invoke(
    "What city did I visit last?"
)

[Document(page_content='In 2022, I visited New York'),
Document(page_content='In 2021, I visited New Orleans')]

我们现在可以创建用于问答的链

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI

这是基础问答链的设置。

In [ ]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retriever = vectorstore.as_retriever()

在这里我们将检索器标记为具有可配置字段。所有向量存储检索器都拥有`search_kwargs`作为字段。这只是一个字典，包含向量存储特定的字段。

In [ ]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

我们现在可以使用可配置的检索器来创建链

In [ ]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("What city did I visit last?")

Paris

我们现在可以通过可配置选项来调用链。`search_kwargs` 是可配置字段的标识符，其值为 Astra DB 使用的搜索参数。

In [ ]:
chain.invoke(
    "What city did I visit last?",
    config={"configurable": {"search_kwargs": {"body_search": "new"}}},
)

New York